# Importing Libraries and loading in data

In [2]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import glob
import os
import datetime
import pandas as pd
from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio

In [2]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    time_str = os.path.basename(filename)[17:-13]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

In [3]:
files = glob.glob(r'D:\MAIAC_Data\Europe\h00v01\Projected\2014\*_PM25.tif')

In [4]:
list_of_das = map(maiac_file_to_da, files)
MAIAC_PM25 = xr.concat(list_of_das, 'time')

# Validating Southampton Centre
## Extracting data at the ground site

In [5]:
a = MAIAC_PM25.attrs['affine']
a = rasterio.Affine.from_gdal(*a)
~a * (442579, 112248)

(1106.3836110057598, 1048.137442006313)

In [6]:
folders = glob.glob(r'D:\MAIAC_Data\Europe\h00v01\Projected\20*')

In [7]:
for folder in folders:
    files = glob.glob(r'{fname}\*_PM25.tif'.format(fname=folder))
    list_of_das = map(maiac_file_to_da, files)
    MAIAC_PM25 = xr.concat(list_of_das, 'time')
    reordered_MAIAC_PM25 = MAIAC_PM25.isel(time=np.argsort(MAIAC_PM25.time))
    Measurements = reordered_MAIAC_PM25.isel(x=1106, y=1048).dropna('time').to_dataframe(name='MAIAC_PM25')
    Measurements.to_csv(r'D:\Annies_Dissertation\Methodology\Validation\AURN\Southampton\Soton_MAIAC_PM25.csv', mode='a')
    print('Processed {fname}'.format(fname=folder))

Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2001
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2002
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2003
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2004
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2005
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2006
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2007
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2008
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2009
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2010
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2011
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2012
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2013
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2014


## Comparing the MAIAC and groundbased data

In [15]:
AURN_Soton = pd.read_csv(r'D:\Annies_Dissertation\Methodology\Validation\AURN\Southampton\Southampton2008_2014.csv', 
                         parse_dates = [['Date', 'Time']])

In [16]:
AURN_Soton[:10]

,Date_Time,PM2.5_Soton_Centre,Status
0,06/11/2008 01:00:00,26,V ugm-3 (TEOM FDMS)
1,06/11/2008 02:00:00,18,V ugm-3 (TEOM FDMS)
2,06/11/2008 03:00:00,19,V ugm-3 (TEOM FDMS)
3,06/11/2008 04:00:00,22,V ugm-3 (TEOM FDMS)
4,06/11/2008 05:00:00,25,V ugm-3 (TEOM FDMS)
5,06/11/2008 06:00:00,36,V ugm-3 (TEOM FDMS)
6,06/11/2008 07:00:00,39,V ugm-3 (TEOM FDMS)
7,06/11/2008 08:00:00,37,V ugm-3 (TEOM FDMS)
8,06/11/2008 09:00:00,41,V ugm-3 (TEOM FDMS)
9,06/11/2008 10:00:00,32,V ugm-3 (TEOM FDMS)


In [17]:
MAIAC_Soton = pd.read_csv(r'D:\Annies_Dissertation\Methodology\Validation\AURN\Southampton\Soton_MAIAC_PM25.csv')

In [18]:
MAIAC_Soton[:10]

,time,x,y,MAIAC_PM25
0,25/02/2000 10:50,442096.9763,112420.7018,23.026955
1,28/02/2000 11:25,442096.9763,112420.7018,36.062874
2,03/03/2000 12:35,442096.9763,112420.7018,20.079552
3,12/03/2000 12:30,442096.9763,112420.7018,25.816568
4,08/04/2000 10:35,442096.9763,112420.7018,27.164116
5,10/04/2000 10:20,442096.9763,112420.7018,14.609890
6,10/04/2000 12:00,442096.9763,112420.7018,23.419872
7,01/05/2000 12:20,442096.9763,112420.7018,16.667433
8,20/05/2000 11:10,442096.9763,112420.7018,0.000000
9,25/05/2000 11:30,442096.9763,112420.7018,0.000000


In [19]:
merged_inner = AURN_Soton.merge(MAIAC_Soton, left_on='Date_Time', right_on='time', how='inner')

In [21]:
merged_inner

,Date_Time,PM2.5_Soton_Centre,Status,time,x,y,MAIAC_PM25


# Validating Bournemouth

In [8]:
a = MAIAC_PM25.attrs['affine']
a = rasterio.Affine.from_gdal(*a)
~a * (412322, 93343)

(1082.3040538889738, 1063.182688636608)

In [9]:
for folder in folders:
    files = glob.glob(r'{fname}\*_PM25.tif'.format(fname=folder))
    list_of_das = map(maiac_file_to_da, files)
    MAIAC_PM25 = xr.concat(list_of_das, 'time')
    reordered_MAIAC_PM25 = MAIAC_PM25.isel(time=np.argsort(MAIAC_PM25.time))
    Measurements = reordered_MAIAC_PM25.isel(x=1082, y=1063).dropna('time').to_dataframe(name='MAIAC_PM25')
    Measurements.to_csv(r'D:\Annies_Dissertation\Methodology\Validation\AURN\Bournemouth\Bmth_MAIAC_PM25.csv', mode='a')
    print('Processed {fname}'.format(fname=folder))

Processed D:\MAIAC_Data\Europe\h00v01\Projected\2000
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2001
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2002
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2003
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2004
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2005
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2006
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2007
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2008
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2009
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2010
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2011
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2012
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2013
Processed D:\MAIAC_Data\Europe\h00v01\Projected\2014


In [23]:
AURN_Bmth = pd.read_csv(r'D:\Annies_Dissertation\Methodology\Validation\AURN\Bournemouth\Bournemouth2008_2014.csv')